# 多维异常检测算法仿真（算法实践）

## 1、数据采集

> 读取 CPU\磁盘读\磁盘写\网络出口\网络入口\内存等监控指标，数据来源：广西科大数据（2017.1-2017.2）

In [25]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(15, 6))
from dateutil.parser import parse
from sklearn.metrics import r2_score

In [26]:
vCpuUsage = pd.read_excel('../ECUST data/Guangxi university data 20170228/CPU_20170228171221.xlsx',converters={u'时间':parse})
vDiskRead = pd.read_excel('../ECUST data/Guangxi university data 20170228/DiskRead_20170228171404.xlsx',converters={u'时间':parse})
vDiskWrite = pd.read_excel('../ECUST data/Guangxi university data 20170228/DiskWrite_20170228171432.xlsx',converters={u'时间':parse})
vNwEgress = pd.read_excel('../ECUST data/Guangxi university data 20170228/NwEgress_20170228171526.xlsx',converters={u'时间':parse})
vNwIngress = pd.read_excel('../ECUST data/Guangxi university data 20170228/NwIngress_20170228171623.xlsx',converters={u'时间':parse})
vMemUsage = pd.read_excel('../ECUST data/Guangxi university data 20170228/Memory_20170228171333.xlsx',converters={u'时间':parse})

## 2.数据预处理

In [27]:
import time
start=time.time()

### 修改index和columns
for var in (vCpuUsage,vDiskRead,vDiskWrite,vNwEgress,vNwIngress,vMemUsage):
    var.rename(columns={u'资源':'vres',u'类型':'vtype',u'时间':'vtime',u'最大值':'vmax',u'最小值':'vmin',u'平均值':'vavg',u'单位':'vunit'},
               inplace = True)
    if 'vtime' in var.columns.values:
        var.set_index('vtime',inplace=True) 

### 初步探索时间序列数据,形成待分析多维数据矩阵X
X = pd.concat([vCpuUsage.to_period('Min').vavg,
               vDiskRead.to_period('Min').vavg,
               vDiskWrite.to_period('Min').vavg,
               vNwEgress.to_period('Min').vavg,
               vNwIngress.to_period('Min').vavg,
               vMemUsage.to_period('Min').vavg],axis=1,keys=['vCpuUsage','vDiskRead','vDiskWrite','vNwEgress','vNwIngress','vMemUsage'])

### 对缺失数据进行插值处理
#设定初始值后，对NaN进行线性插值
X.ix[0,X.ix[0].isnull()]=0
X.interpolate(method='time',inplace=True)

### 对CPU 0值数据进行填充
for i in range(1,len(X.vCpuUsage)):
    if X.vCpuUsage[i]==0:
        X.vCpuUsage[i] = X.vCpuUsage[i-1] 

#保留原始值
X_Original=X

### 无量纲化
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

X_MinMaxScaler=MinMaxScaler().fit(X_Original)
X = DataFrame(X_MinMaxScaler.transform(X_Original),index=X_Original.index,columns=X_Original.columns)

#区间缩放后再将均值0化，这主要是由于部分算法会自行对均值进行处理（比如pca的transform），为避免算法理解上的干扰，调整均值为0
X_mean=X.mean()
X = X-X_mean

## 3、获取PCA残差

In [28]:
from sklearn.decomposition import PCA

#指定主成分的方差和所占的最小比例阈值为0.85
X_pca=PCA(n_components=0.85).fit(X)
X_pca_T=DataFrame(X_pca.transform(X),index=X.index)

### 形成残差空间
X_pca_recover=DataFrame(np.dot(X_pca_T,X_pca.components_),index=X.index,columns=X.columns)
X_pca_residual=X-X_pca_recover

## 4、基于残差空间的PCA异常检测算法应用

In [29]:
from scipy import stats

def my_kde_bandwidth(obj, fac=1./2):
    """We use Scott's Rule, multiplied by a constant factor."""
    return np.power(obj.n, -1./(obj.d+4)) * fac

def get_threshold_of_scipy_kde(kde,start,step=1,confidence=0.997):
    """get threshold by confidence"""
    i = start
    cumsum = kde.integrate_box_1d(-np.inf, start)
    while True:
        if cumsum >= confidence:
            break
        cumsum = cumsum + kde.integrate_box_1d(i, i+step)
        i = i + step        
    return i

#提取异常
X_pca_residual_pca=PCA(n_components=0.85).fit(X_pca_residual)
X_pca_residual_pca_T=DataFrame(X_pca_residual_pca.transform(X_pca_residual),index=X_pca_residual.index)
X_pca_residual_pca_recover=DataFrame(np.dot(X_pca_residual_pca_T,X_pca_residual_pca.components_),index=X_pca_residual.index,columns=X_pca_residual.columns)

#计算T2统计量（实际上是各样本得分值先平方再按特征值归一化后求和）
X_pca_residual_pca_T2=Series(np.sum(X_pca_residual_pca_T**2/X_pca_residual_pca.explained_variance_,axis=1),index=X_pca_residual.index)
#计算SPE统计量
X_pca_residual_pca_SPE=Series(np.sum((X_pca_residual-X_pca_residual_pca_recover)**2,axis=1),index=X_pca_residual.index)

#通过概率密度函数求解概率时的累加步长设置(中位数与最大值距离100步)
X_pca_residual_pca_T2_pdf_step=(X_pca_residual_pca_T2.max()-X_pca_residual_pca_T2.median())/100
X_pca_residual_pca_SPE_pdf_step=(X_pca_residual_pca_SPE.max()-X_pca_residual_pca_SPE.median())/100

#kde及阈值估计
X_pca_residual_pca_T2_scipy_kde=stats.gaussian_kde(X_pca_residual_pca_T2, bw_method=my_kde_bandwidth)
X_pca_residual_pca_SPE_scipy_kde=stats.gaussian_kde(X_pca_residual_pca_SPE, bw_method=my_kde_bandwidth)
X_pca_residual_pca_T2_threshold=get_threshold_of_scipy_kde(X_pca_residual_pca_T2_scipy_kde,X_pca_residual_pca_T2.min(),step=X_pca_residual_pca_T2_pdf_step,confidence=0.997)
X_pca_residual_pca_SPE_threshold=get_threshold_of_scipy_kde(X_pca_residual_pca_SPE_scipy_kde,X_pca_residual_pca_SPE.min(),step=X_pca_residual_pca_SPE_pdf_step,confidence=0.997)

#输出异常时间段
X_pca_residual_pca_T2_anomaly=X_pca_residual_pca_T2[X_pca_residual_pca_T2>X_pca_residual_pca_T2_threshold].index
indice=pd.Series([True]+list(np.diff(X_pca_residual_pca_T2_anomaly)>10))
X_pca_residual_pca_T2_anomaly_start=X_pca_residual_pca_T2_anomaly[indice].tolist()
X_pca_residual_pca_T2_anomaly_end=X_pca_residual_pca_T2_anomaly[indice.shift(-1).fillna(False)].tolist()
X_pca_residual_pca_T2_anomaly_end.append(X_pca_residual_pca_T2_anomaly[-1])
print('anomal periods detected by T2 metric are: ')
for each in zip(X_pca_residual_pca_T2_anomaly_start,X_pca_residual_pca_T2_anomaly_end):    
    print(each)

X_pca_residual_pca_SPE_anomaly=X_pca_residual_pca_SPE[X_pca_residual_pca_SPE>X_pca_residual_pca_SPE_threshold].index
indice=pd.Series([True]+list(np.diff(X_pca_residual_pca_SPE_anomaly)>10))
X_pca_residual_pca_SPE_anomaly_start=X_pca_residual_pca_SPE_anomaly[indice].tolist()
X_pca_residual_pca_SPE_anomaly_end=X_pca_residual_pca_SPE_anomaly[indice.shift(-1).fillna(False)].tolist()
X_pca_residual_pca_SPE_anomaly_end.append(X_pca_residual_pca_SPE_anomaly[-1])

print('anomal periods detected by SPE metric are: ')
for each in zip(X_pca_residual_pca_SPE_anomaly_start,X_pca_residual_pca_SPE_anomaly_end):
    print(each)

print('算法用时：',time.time()-start,'s')

anomal periods detected by T2 metric are: 
(Period('2017-01-18 09:50', 'T'), Period('2017-01-18 10:09', 'T'))
(Period('2017-01-18 10:44', 'T'), Period('2017-01-18 10:47', 'T'))
(Period('2017-01-20 12:36', 'T'), Period('2017-01-20 12:55', 'T'))
(Period('2017-01-21 15:23', 'T'), Period('2017-01-21 15:43', 'T'))
(Period('2017-02-20 10:21', 'T'), Period('2017-02-20 10:35', 'T'))
(Period('2017-02-20 10:56', 'T'), Period('2017-02-20 11:16', 'T'))
(Period('2017-02-20 11:43', 'T'), Period('2017-02-20 12:00', 'T'))
(Period('2017-02-20 12:14', 'T'), Period('2017-02-20 12:23', 'T'))
(Period('2017-02-20 12:34', 'T'), Period('2017-02-20 13:02', 'T'))
(Period('2017-02-20 13:26', 'T'), Period('2017-02-20 13:44', 'T'))
(Period('2017-02-20 15:21', 'T'), Period('2017-02-20 15:27', 'T'))
(Period('2017-02-20 16:25', 'T'), Period('2017-02-20 16:37', 'T'))
(Period('2017-02-20 18:13', 'T'), Period('2017-02-20 18:45', 'T'))
(Period('2017-02-20 19:04', 'T'), Period('2017-02-20 19:19', 'T'))
(Period('2017-02-22

## 5、在线应用

### 5.1、数据采集

In [70]:
testX=X_Original.ix['2017-01-18 08:50':'2017-01-18 10:50']

### 5.2、测试

In [71]:
start=time.time()

#数据预处理
testX = DataFrame(X_MinMaxScaler.transform(testX),index=testX.index,columns=testX.columns)-X_mean

#获取残差空间
testX_pca_T=DataFrame(X_pca.transform(testX),index=testX.index)
testX_pca_recover=DataFrame(np.dot(testX_pca_T,X_pca.components_),index=testX.index,columns=testX.columns)
testX_pca_residual=testX-testX_pca_recover

#提取异常
testX_pca_residual_pca_T=DataFrame(X_pca_residual_pca.transform(testX_pca_residual),index=testX_pca_residual.index)
testX_pca_residual_pca_recover=DataFrame(np.dot(testX_pca_residual_pca_T,X_pca_residual_pca.components_),index=testX_pca_residual.index,columns=testX_pca_residual.columns)

#计算T2统计量（实际上是各样本得分值先平方再按特征值归一化后求和）
testX_pca_residual_pca_T2=Series(np.sum(testX_pca_residual_pca_T**2/X_pca_residual_pca.explained_variance_,axis=1),index=testX_pca_residual.index)
#计算SPE统计量
testX_pca_residual_pca_SPE=Series(np.sum((testX_pca_residual-testX_pca_residual_pca_recover)**2,axis=1),index=testX_pca_residual.index)

#输出异常时间段
testX_pca_residual_pca_T2_anomaly=testX_pca_residual_pca_T2[testX_pca_residual_pca_T2>X_pca_residual_pca_T2_threshold].index
if len(testX_pca_residual_pca_T2_anomaly)!=0:
    indice=pd.Series([True]+list(np.diff(testX_pca_residual_pca_T2_anomaly)>10))
    testX_pca_residual_pca_T2_anomaly_start=testX_pca_residual_pca_T2_anomaly[indice].tolist()
    testX_pca_residual_pca_T2_anomaly_end=testX_pca_residual_pca_T2_anomaly[indice.shift(-1).fillna(False)].tolist()
    testX_pca_residual_pca_T2_anomaly_end.append(testX_pca_residual_pca_T2_anomaly[-1])
    print('anomal periods detected by T2 metric are: ')
    for each in zip(testX_pca_residual_pca_T2_anomaly_start,testX_pca_residual_pca_T2_anomaly_end):    
        print(each)
else:
    print('there\'s no anomal periods detected by T2 metric ')

testX_pca_residual_pca_SPE_anomaly=testX_pca_residual_pca_SPE[testX_pca_residual_pca_SPE>X_pca_residual_pca_SPE_threshold].index
if len(testX_pca_residual_pca_SPE_anomaly)!=0:
    indice=pd.Series([True]+list(np.diff(testX_pca_residual_pca_SPE_anomaly)>10))
    testX_pca_residual_pca_SPE_anomaly_start=testX_pca_residual_pca_SPE_anomaly[indice].tolist()
    testX_pca_residual_pca_SPE_anomaly_end=testX_pca_residual_pca_SPE_anomaly[indice.shift(-1).fillna(False)].tolist()
    testX_pca_residual_pca_SPE_anomaly_end.append(testX_pca_residual_pca_SPE_anomaly[-1])
    print('anomal periods detected by SPE metric are: ')
    for each in zip(testX_pca_residual_pca_SPE_anomaly_start,testX_pca_residual_pca_SPE_anomaly_end):
        print(each)
else:
    print('there\'s no anomal periods detected by SPE metric ')
    
print('算法用时：',time.time()-start,'s')

anomal periods detected by T2 metric are: 
(Period('2017-01-18 09:50', 'T'), Period('2017-01-18 10:09', 'T'))
(Period('2017-01-18 10:44', 'T'), Period('2017-01-18 10:47', 'T'))
anomal periods detected by SPE metric are: 
(Period('2017-01-18 09:53', 'T'), Period('2017-01-18 09:54', 'T'))
算法用时： 0.01584482192993164 s
